In [1]:
from google.cloud import storage
import os
from io import StringIO # if going with no saving csv file
import gcsfs
import dask.dataframe as dd
from dask.delayed import delayed
from pathlib import Path
import pandas as pd
import os

In [2]:
data_dir = Path(os.getenv('APP_DIR') + '/data')

userlogin_datadir = Path(data_dir, 'nanohub_metrics/userlogin')
jos_users_datadir = Path(data_dir, 'nanohub/jos_users')

In [3]:
%load_ext memory_profiler

In [4]:
%%timeit
%%memit

full_df = pd.concat(
    pd.read_parquet(parquet_file)
    for parquet_file in userlogin_datadir.glob('*.parquet.gzip')
)

peak memory: 33799.30 MiB, increment: 33632.58 MiB
peak memory: 48501.38 MiB, increment: 32570.80 MiB
peak memory: 48474.27 MiB, increment: 33110.76 MiB
peak memory: 48406.99 MiB, increment: 33012.30 MiB
peak memory: 48437.77 MiB, increment: 32949.85 MiB
peak memory: 48359.65 MiB, increment: 32867.53 MiB
peak memory: 48330.62 MiB, increment: 32762.37 MiB
peak memory: 48412.21 MiB, increment: 32842.75 MiB
1min 10s ± 852 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
print(full_df.head())

                     datetime user  uidnumber              ip  action
id                                                                   
264497646 2018-02-19 04:39:48    -          0   157.55.39.123  detect
264497647 2018-02-19 04:39:49    -          0  14.139.181.229  detect
264497648 2018-02-19 04:39:49    -          0   46.229.168.80  detect
264497649 2018-02-19 04:39:50    -          0   118.185.164.5  detect
264497650 2018-02-19 04:39:51    -          0  14.139.181.229  detect


In [6]:
%%timeit
%%memit


filenames = userlogin_datadir.glob('*.parquet.gzip')
dfs = [delayed(pd.read_parquet)(fn) for fn in filenames]

dask_df = dd.from_delayed(dfs) 

peak memory: 15817.86 MiB, increment: 183.86 MiB
peak memory: 15862.20 MiB, increment: 44.34 MiB
peak memory: 15878.77 MiB, increment: 16.57 MiB
peak memory: 15842.35 MiB, increment: -12.44 MiB
peak memory: 15829.26 MiB, increment: 9.00 MiB
peak memory: 15822.76 MiB, increment: 26.04 MiB
peak memory: 15841.28 MiB, increment: 40.35 MiB
peak memory: 15817.43 MiB, increment: -0.79 MiB
520 ms ± 18 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
print(dask_df.head())
df = dask_df.compute()

                     datetime user  uidnumber              ip  action
id                                                                   
264497646 2018-02-19 04:39:48    -          0   157.55.39.123  detect
264497647 2018-02-19 04:39:49    -          0  14.139.181.229  detect
264497648 2018-02-19 04:39:49    -          0   46.229.168.80  detect
264497649 2018-02-19 04:39:50    -          0   118.185.164.5  detect
264497650 2018-02-19 04:39:51    -          0  14.139.181.229  detect


In [ ]:
%%timeit
%%memit

from multiprocessing import Pool

def reader(filename):
    return pd.read_parquet(filename)

def main(data_dir):
    pool = Pool(16) # number of cores you want to use
    file_list = data_dir.glob('*.parquet.gzip')
    df_list = pool.map(reader, file_list) #creates a list of the loaded df's
    return pd.concat(df_list) # concatenates all the df's into a single df

df = main(userlogin_datadir)

In [ ]:
print(df.head())
print(df.tail())